In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))
from keras.optimizers import SGD, Adam
import keras
from keras.models import Sequential
from keras.utils import to_categorical
from keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, Convolution2D

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import interpolate
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
pd.options.mode.chained_assignment = None
from scipy.interpolate import griddata
# Any results you write to the current directory are saved as output.

In [ ]:
train_data = pd.read_csv('../input/Train.csv')
test_data = pd.read_csv('../input/Test.csv')

In [ ]:
labels = train_data['label']
train = pd.concat([train_data.iloc[:, 368:len(train_data.columns)-1]], axis=1)
test = pd.concat([test_data.iloc[:, 368:len(test_data.columns)]], axis=1)
train_un = pd.concat([train_data.iloc[:, 368:len(train_data.columns)-1]], axis=1)
test_un = pd.concat([test_data.iloc[:, 368:len(test_data.columns)]], axis=1)

In [ ]:
def interpolation(array):
    x = np.arange(0, array.shape[1])
    y = np.arange(0, array.shape[0])
    array = np.ma.masked_invalid(array)
    xx, yy = np.meshgrid(x, y)
    x1 = xx[~array.mask]
    y1 = yy[~array.mask]
    newarr = array[~array.mask]
    GD1 = griddata((x1, y1), newarr.ravel(),(xx, yy),method='cubic',fill_value=255)
    return GD1.reshape(1024,)

In [ ]:
def imgShow(im):
    res = np.array(im).reshape(32,32)
    plt.imshow(res,interpolation='nearest', cmap = plt.cm.gray)

In [ ]:
imgShow(X_train.iloc[45])

In [ ]:
for i in range(0,test.shape[0]):
    test.iloc[i] = interpolation(np.array(test.iloc[i]).reshape(32,32))

In [ ]:
train = train.fillna(255)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, labels)

In [ ]:
test

In [ ]:
X_train = X_train.values.reshape(X_train.shape[0], 32, 32, 1)
X_test = X_test.values.reshape(X_test.shape[0], 32, 32, 1)
y_train = to_categorical(y_train,10)
y_test = to_categorical(y_test,10)

In [ ]:
imgShow(test.iloc[1])

In [ ]:
# build and compile the model  (roughly following the VGG paper)
import tensorflow as tf
import numpy as np                                
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
import keras as k
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.regularizers import l2
import h5py
from keras.models import load_model
from keras.preprocessing.image import ImageDataGenerator
#reg=l2(1e-4)   # L2 or "ridge" regularisation
reg=None
num_filters=32
ac='relu'
adm=Adam(lr=0.001,decay=0, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
opt=adm
drop_dense=0.5
drop_conv=0

model = Sequential()

model.add(Conv2D(num_filters, (3, 3), activation=ac, kernel_regularizer=reg, input_shape=(32, 32, 1),padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 16x16x3xnum_filters
model.add(Dropout(drop_conv))

model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(2*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 8x8x3x(2*num_filters)
model.add(Dropout(drop_conv))

model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(Conv2D(4*num_filters, (3, 3), activation=ac,kernel_regularizer=reg,padding='same'))
model.add(BatchNormalization(axis=-1))
model.add(MaxPooling2D(pool_size=(2, 2)))   # reduces to 4x4x3x(4*num_filters)
model.add(Dropout(drop_conv))

model.add(Flatten())
model.add(Dense(512, activation=ac,kernel_regularizer=reg))
model.add(BatchNormalization())
model.add(Dropout(drop_dense))
model.add(Dense(10, activation='softmax'))

model.compile(loss='categorical_crossentropy', metrics=['accuracy'],optimizer=opt)

In [ ]:
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train,batch_size=128, epochs=100, validation_data=(X_test, y_test))

In [ ]:
res = model.predict_classes(test_final)

In [ ]:
read = pd.read_csv('../input/sample.csv')
read['label'] = res

In [ ]:
read.to_csv('submission.csv',index=False)